### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [5]:
# Set environment variables from the .env in the local environment
load_dotenv('example.env')

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

# Make sure the API keys were loaded
print(type(nyt_api_key))
print(type(tmdb_api_key))

<class 'str'>
<class 'str'>


### Access the New York Times API

In [8]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:("Movies") AND type_of_material:("Review") AND headline:("love")'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
nyt_base_url = f"{url}&fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&api-key={nyt_api_key}"

# Print the URL
print(nyt_base_url)


https://api.nytimes.com/svc/search/v2/articlesearch.json?&fq=section_name:("Movies") AND type_of_material:("Review") AND headline:("love")&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531&api-key=JwBGKZJxwgxxgOkf7GtUznmgBPuGAXyl


In [9]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0, 1):

    # create query with a page number
    # API results show 10 articles at a time
    query_url = f"{nyt_base_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_url).json()

    # Print the status code
    print(f"Status code: {reviews['status']}")
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)

        # Print the page number that was just retrieved
        print(f"Checked page {page}")
    except:
        # Print the page number that had no results then break from the loop
        print(f"Page {page} had no results")
        break


Status code: OK
Checked page 0


In [12]:
# Preview the first 5 results unformatted
reviews_list[:5]

[{'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html',
  'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.',
  'source': 'The New York Times',
  'headline': {'main': '‘The Attachment Diaries’ Review: Love, Sick',
   'kicker': None,
   'content_kicker': None,
   'print_headline': 'The Attachment Diaries',
   'name': None,
   'seo': None,
   'sub': None},
  'keywords': [{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'},
   {'name': 'creative_works',
    'value': 'The Attachment Diaries (Movie)',
    'rank': 2,
    'major': 'N'},
   {'name': 'persons',
    'value': 'Diment, Valentin Javier',
    'rank': 3,
    'major': 'N'}],
  'pub_date': '2023-05-25T11:00:03+0000',
  'byline': {'original': 'By Jeannette Catsoulis',
   'person': [{'firstname': 'Jeannette',
     'middlename': None,
     'lastname': 'Catsoulis',
     'qualifier': None,
     'title': None,
     'rol

In [13]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

print(json.dumps(reviews_list[:5], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [38]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)

# Preview the DataFrame
reviews_df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'fir

In [39]:
# Test extracting the title from the "headline.main" column
# NOTE: This cuts off the final "t" from "Review: ‘What’s Love Got to Do With It?’ Probably a Lot"
sub_text = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
print(sub_text)
print(reviews_df['headline.main'][1])


0                           The Attachment Diaries
1    What’s Love Got to Do With It?’ Probably a Lo
2                             You Can Live Forever
3                        A Tourist’s Guide to Love
4                          Other People’s Children
5                                   One True Loves
6                   The Lost Weekend: A Love Story
7                               A Thousand and One
8                               Your Place or Mine
9                     Love in the Time of Fentanyl
Name: headline.main, dtype: object
Review: ‘What’s Love Got to Do With It?’ Probably a Lot


In [40]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
# NOTE: This cuts off the final "t" from "Review: ‘What’s Love Got to Do With It?’ Probably a Lot"
reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

# Preview the DataFrame
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The

In [41]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list) -> str:
    extracted_keywords: str = ""
    sep: str = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword: str = f"{sep}{item['name']}: {item['value']}" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
        sep = "; "
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(lambda keyword_list: extract_keywords(keyword_list))

# Preview the DataFrame
reviews_df


subject: Movies; persons: Kapur, Shekhar; persons: James, Lily; persons: Azmi, Shabana; persons: Thompson, Emma


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies; creative_works: The Attachmen...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies; persons: Kapur, Shekhar; pers...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies; creative_works: You Can Live ...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies; creative_works: A Tourist's G...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies; persons: Zlotowski, Rebecca; ...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies; persons: Bracey, Luke (1989- ...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs; creat...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies; creative_works: A Thousand an...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,subject: Movies; creative_works: Your Place or...,2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Ti

In [42]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = reviews_df['title'].to_list()

# Preview the list
title_list


['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl']

### Access The Movie Database API

In [65]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [70]:
# Test The Movie Database query
tmdb_query_url = url + title_list[0] + tmdb_key_string
print(tmdb_query_url)

# Make the query
tmdb_response = requests.get(tmdb_query_url).json()
movie_id = tmdb_response['results'][0]['id']
print(f"movie_id={movie_id}")
#print(json.dumps(tmdb_response, indent=4))

# Test The Movie details query
tmdb_details_query_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
print(tmdb_details_query_url)
tmdb_details_response = requests.get(tmdb_details_query_url).json()
print(json.dumps(tmdb_details_response, indent=4))
#print(type(tmdb_details_response))

print(tmdb_details_response['genres'])
print(tmdb_details_response['spoken_languages'])
print(tmdb_details_response['production_countries'])

# Extract the genre names into a list
genre_names = [genre['name'] for genre in tmdb_details_response['genres']]
print(genre_names)

# Extract the spoken_languages' English name into a list
spoken_languages = [lang['english_name'] for lang in tmdb_details_response['spoken_languages']]
print(spoken_languages)

# Extract the production_countries' name into a list
production_countries = [country['name'] for country in tmdb_details_response['production_countries']]
print(production_countries)

# Need to clean up the data beofre loading into DataFrame
# tmdb_details_response_df = pd.DataFrame(tmdb_details_response)
# tmdb_details_response_df

https://api.themoviedb.org/3/search/movie?query=The Attachment Diaries&api_key=3099e29947671d22110ad06766c5bb88
movie_id=743040
https://api.themoviedb.org/3/movie/743040?api_key=3099e29947671d22110ad06766c5bb88
{
    "adult": false,
    "backdrop_path": null,
    "belongs_to_collection": null,
    "budget": 0,
    "genres": [
        {
            "id": 18,
            "name": "Drama"
        },
        {
            "id": 9648,
            "name": "Mystery"
        },
        {
            "id": 53,
            "name": "Thriller"
        },
        {
            "id": 27,
            "name": "Horror"
        }
    ],
    "homepage": "",
    "id": 743040,
    "imdb_id": "tt11359158",
    "origin_country": [
        "ES"
    ],
    "original_language": "es",
    "original_title": "El apego",
    "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she pro

In [71]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Create a  list of desired columns
tmdb_cols: list[str] = ["title", "original_title", "budget", 
                        "genres", "original_language", "spoken_languages", 
                        "homepage", "overview", "popularity", 
                        "runtime", "revenue", "release_date", 
                        "vote_average", "vote_count", "production_countries"]

# Loop through the titles
for title in title_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        time.sleep(1)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    tmdb_query_url = url + title + tmdb_key_string
    tmdb_response = requests.get(tmdb_query_url).json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = tmdb_response['results'][0]['id']

        # Make a request for a the full movie details
        tmdb_details_query_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"

        # Execute "GET" request with url
        tmdb_details_response = requests.get(tmdb_details_query_url).json()
        
        # Extract the genre names into a list
        genre_names = [genre['name'] for genre in tmdb_details_response['genres']]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [lang['english_name'] for lang in tmdb_details_response['spoken_languages']]

        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in tmdb_details_response['production_countries']]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        # NOTE: tmdb_details_response is a dictionary but has more columns than we need
        #       so we need to clean up the data
        tmdb_details_response['genres'] = genre_names
        tmdb_details_response['spoken_languages'] = spoken_languages
        tmdb_details_response['production_countries'] = production_countries

        # Create a dictionary with the relevant data
        tmdb_movie = {}
        for col in tmdb_cols:
            tmdb_movie[col] = tmdb_details_response[col]

        tmdb_movies_list.append(tmdb_movie)

        # Print out the title that was found
        print(f"Found {title}")

    except Exception as e:
        #print(f"{title} was not found. Cause: {type(e).__name__}: {e}")
        print(f"{title} was not found.")



Found The Attachment Diaries
What’s Love Got to Do With It?’ Probably a Lo was not found. IndexError: list index out of range
Found You Can Live Forever
Found A Tourist’s Guide to Love
Found Other People’s Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl


In [72]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "original_language": "es",
        "spoken_languages": [
            "Spanish"
        ],
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 6.179,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "production_countries": [
            "Argentina"
        ]
 

In [73]:
# Convert the results to a DataFrame
tmdb_movies_list_df = pd.DataFrame(tmdb_movies_list)

# Preview the DataFrame
tmdb_movies_list_df

,title,original_title,budget,genres,original_language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",6.179,102,0,2021-10-07,3.000,4,[Argentina]
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, French]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",25.936,96,15055,2023-03-24,6.622,37,"[Canada, United States of America]"
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy]",en,"[English, Vietnamese]",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",12.070,96,0,2023-04-21,6.289,161,[United States of America]
3,Other People's Children,Les Enfants des autres,0,"[Drama, Comedy]",fr,"[French, English]",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",8.708,104,84178,2022-09-21,6.823,189,[France]
4,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,"[English, Spanish]",,Emma and Jesse are living the perfect life tog...,10.860,100,37820,2023-04-07,6.507,73,"[Czech Republic, United States of America]"
5,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,[Documentary],en,[English],https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,3.114,95,77145,2023-04-13,6.000,2,[United States of America]
6,A Thousand and One,A Thousand and One,0,"[Drama, Crime]",en,"[English, Spanish, Portuguese]",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,9.188,116,3395595,2023-03-31,6.900,88,[United States of America]
7,Your Place or Mine,Your Place or Mine,0,"[Romance, Comedy]",en,[English],https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,20.736,109,0,2023-02-10,6.281,709,[United States of America]
8,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,[Documentary],en,[English],,An intimate portrait of a community fighting t...,0.773,85,0,2023-02-03,0.000,0,"[Canada, United States of America]"


### Merge and Clean the Data for Export

In [94]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_movies_df = pd.merge(tmdb_movies_list_df, reviews_df, how="inner", on="title")

# Preview the DataFrame
merged_movies_df

,title,original_title,budget,genres,original_language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",6.179,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, French]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",25.936,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,"[English, Spanish]",,Emma and Jesse are living the perfect life tog...,10.860,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,[Documentary],en,[English],https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,3.114,95,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,"[Drama, Crime]",en,"[English, Spanish, Portuguese]",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,9.188,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
5,Your Place or Mine,Your Place or Mine,0,"[Romance, Comedy]",en,[English],https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,20.736,109,...,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
6,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,[Documentary],en,[English],,An intimate portrait of a community fighting t...,0.773,85,...,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None


In [95]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
cols_to_fix = ["genres", "spoken_languages", "production_countries"]

# Create a list of characters to remove
chars_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for col in cols_to_fix:

    # Convert the column to type 'str'
    merged_movies_df[col] = merged_movies_df[col].astype(str)

    # Loop through characters to remove
    for char in chars_to_remove:
        #print(f"col={col}, char={char}, type={merged_movies_df[col].dtype}")
        merged_movies_df[col] = merged_movies_df[col].str.replace(char, "")

# TODO: Rename columns {"genres": "genre", "original_language": "language"}

# Display the fixed DataFrame
merged_movies_df.head()


,title,original_title,budget,genres,original_language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",6.179,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",25.936,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,"English, Spanish",,Emma and Jesse are living the perfect life tog...,10.860,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,English,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,3.114,95,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,"Drama, Crime",en,"English, Spanish, Portuguese",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,9.188,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [96]:
# Drop "byline.person" column
merged_movies_df.drop(columns=["byline.person"], inplace=True)

# Preview the DataFrame
merged_movies_df.head()


,title,original_title,budget,genres,original_language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",6.179,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",25.936,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,"English, Spanish",,Emma and Jesse are living the perfect life tog...,10.860,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,English,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,3.114,95,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,"Drama, Crime",en,"English, Spanish, Portuguese",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,9.188,116,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None


In [98]:
# Delete duplicate rows and reset index
unique_movies_df = merged_movies_df.drop_duplicates(keep="first").reset_index(drop=True)

# Preview the DataFrame
unique_movies_df.head()

,title,original_title,budget,genres,original_language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",6.179,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",25.936,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,"English, Spanish",,Emma and Jesse are living the perfect life tog...,10.860,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,English,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,3.114,95,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,"Drama, Crime",en,"English, Spanish, Portuguese",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,9.188,116,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None


In [99]:
# Export data to CSV without the index
pd.DataFrame(unique_movies_df).to_csv("output/tmdb_nyt_movie_reviews.csv", index=False)
